# The ERT

It is common, when benchmarking optimization techniques, to introduce the concept of expected run time (ERT). Here we take the same angle and consider **symbolic regression** as an optimisation problem where a model is seeked that minimizes its prediction errors. To benchmark different techniques aimed at solving the very same problem we thus consider the ERT as our main indicator. 

The expected run time, for a **symbolic regression** problem, can be defined as the expected value of $n$, the number of phenotype evaluations made before finding a model having a loss below a fixed threshold. 

To compute the ERT we run *n_trials* independent trials and record, in each one, the number of function evaluations (phenotype evaluations) needed to reach a target *f_tol*. 
If *max_feval* are consumed without reaching *f_tol* we mark the trial as failed. 
We need to introduce *max_feval* and, essentially, restart the trial, as otherwise we would risk to have runs converging in infinite time. 
We can accomodate easily for the presence of restarts in the definition of the final ERT:

$$
ERT = \mathbb E(n) = \frac {\sum_{i=0}^{n_{trials}} n_i}{n_{success}}
$$

We argue that the above indicator, albeit depening on *f_tol* and *max_feval* is the only one that allows for a fair comparison between symbolic regression approaches. Lets see how to compute it for a few problems in the *dsyre* gym.

We start with the usual imports ...

In [1]:
import pydsyre as dsy
import numpy as np
import pygmo as pg

from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
# We import the problem:
xs, ys = dsy.gym.generate_P1()
xs = np.array(xs)
ys = np.array(ys)

We define some hyperparameters:

In [3]:
length = 20 # The number of triplets used
max_mut = 15 # The maximum number of mutations
ncons = 1 # The number of constants in the formula
popsize = 4 # The population size

... and the ERT computation settings:

In [4]:
n_trials = 200 # Number of independent trials
max_feval = 1000 # restart parameter
ftol = 1e-8 # target threshold
gen = max_feval // popsize

We instantiate the pagmo problem and algorithm:

In [5]:
## Instantiate the UDP
udp = dsy.sr_problem(xs = xs, 
                     ys = ys, 
                     length = length,
                     kernels = ["sum", "mul", "diff", "inv"],
                     ncons = ncons, 
                     multi_objective = False);
                     
## Instantiate the UDA
uda = dsy.mes4dsyre(gen = gen, max_mut = max_mut, ftol = ftol)

## Pagmo problem
prob = pg.problem(udp)
## Pagmo algorithm
algo = pg.algorithm(uda)

And we perfrom the *n_trials* runs, computing at the end the ERT.

In [6]:
n_success = 0
ERT=0.
for j in range(n_trials):
    # Pagmo population
    pop = pg.population(prob, popsize)
    # Evolution!
    pop = algo.evolve(pop)
    ERT += pop.problem.get_fevals()
    if (pop.champion_f[0] < ftol):
        n_success+=1
        print(".", end="", flush=True)
    else:
        print("x", end="", flush=True)
        
if (n_success > 0):
    print(f"\n\nERT is: {ERT / n_success}\n")
    print(f"Successful runs: {n_success}\n")
else:
    print("\n\nNo success, adjust ftol or max_feval?\n")
    

...xx.x.x.x.xx...x.x.xx.xx..xxx.x....x.xx..x.xxx..xxxxxxx...x.xx.x..xx...xxx.xxx.xx.x.xxxx.x.xx.xx.x.x.xx.x.xxxxx..xxxx.xxx...x.xx.xx........x..xxxx..x.xxxxx..xx....x.xx.x....xx.xx.xx....xxxxxxx.x..xx

ERT is: 1706.5333333333333

Successful runs: 90

